## Q1.

Add methods `__iter__` to your project Time Series class to iterate over values, a method `itertimes` to iterate over times, a method `itervalues` to iterate over values, and a method `iteritems` to iterate over time-value pairs. (This is a similar interface to python dictionaries). To test these, check both the types of the results and the answers you expect.

In [14]:
#your code here
from TimeSeries import TimeSeries

ts = TimeSeries(range(5),range(1,11,2))
print(type(ts.itervalues()))
print(type(ts.itertimes()))
for i in ts.itervalues():
    print(i)
    
for i in ts.iteritems():
    print(i)

<class 'TimeSeries.TimeSeriesIterator'>
<class 'TimeSeries.TimeSeriesIterator'>
1
3
5
7
9
(0, 1)
(1, 3)
(2, 5)
(3, 7)
(4, 9)


## Q2.

An online mean and standard deviation algorithm.

Below is a function to generate a potentially infinite stream of 1-D data.

In [1]:
from random import normalvariate, random
from itertools import count
def make_data(m, stop=None):
    for _ in count():
        if stop and _ > stop:
            break
        yield 1.0e09 + normalvariate(0, m*random() )
        

Here is an implementation of an online mean algorithm..see http://www.johndcook.com/blog/standard_deviation/ and the link to http://www.johndcook.com/blog/2008/09/26/comparing-three-methods-of-computing-standard-deviation/ in-between. (Convince yourselves of the formulas...)

In [2]:
def online_mean(iterator):
    n = 0
    mu = 0
    for value in iterator:
        n += 1
        delta = value - mu
        mu = mu + delta/n
        yield mu

We use out generator functions to implement iterators:

In [3]:
g = make_data(5, 10)
list(g)

[999999999.5683017,
 999999999.8597099,
 999999999.6919423,
 999999999.8549432,
 999999998.19146,
 1000000000.2917719,
 1000000002.5967975,
 999999998.089182,
 999999999.267127,
 999999999.9862918,
 1000000000.0086004]

In [4]:
g = online_mean(make_data(5, 100))
print(type(g))
list(g)

<class 'generator'>


[1000000000.0989574,
 1000000001.6179042,
 1000000001.713668,
 1000000000.997457,
 1000000000.6507571,
 999999999.981215,
 999999999.8247719,
 999999999.6106342,
 999999999.6782371,
 999999999.5511203,
 999999999.9309299,
 999999999.9283396,
 1000000000.7886372,
 1000000000.8133472,
 1000000000.5641226,
 1000000000.5411795,
 1000000000.2780862,
 1000000000.4199758,
 1000000000.371202,
 1000000000.3107443,
 1000000000.410782,
 1000000000.3811361,
 1000000000.3562423,
 1000000000.2368083,
 1000000000.1732074,
 1000000000.0924106,
 1000000000.1395311,
 1000000000.1793401,
 1000000000.0511202,
 1000000000.0503074,
 1000000000.0446024,
 1000000000.0124029,
 1000000000.0150445,
 999999999.8237804,
 999999999.8430688,
 999999999.7658578,
 999999999.738714,
 999999999.7023369,
 999999999.7337223,
 999999999.868076,
 999999999.8310958,
 999999999.8312185,
 999999999.8168035,
 999999999.7673956,
 999999999.8716522,
 999999999.8613839,
 999999999.8254385,
 999999999.752184,
 999999999.8018132,
 9

### 2.1

Implement the standard deviation algorithm as a generator function as

```python
def online_mean_dev(iterator):
    BLA BLA
    if n > 1:
        stddev = math.sqrt(dev_accum/(n-1))
        yield (n, value, mu, stddev)
```

In [18]:
# your code here
import math
def online_mean_dev(iterator):
    n=0
    mu=0
    dev_accum = 0
    stddev = 0
    for value in iterator: 
        n += 1
        delta = value - mu
        mu = mu + delta/n
        delta2 = value - mu
        dev_accum += delta*delta2
        if n > 1:
            stddev = math.sqrt(dev_accum/(n-1))
            yield (n, value, mu, stddev)

Here we make 100000 element data, and run this iterator on it (imagine running this on a time-series being slowly read from disk

In [21]:
data_with_stats = online_mean_dev(make_data(5, 10000))

## Q3.

Let's do Anomaly detection. Write a routine `is_ok`:

```python
def is_ok(level, t)
```

which takes a tuple like the one yielded by your code above and returns True if the value is inbetween `level`$*\sigma$ of the mean.

In [31]:
#your code here
def is_ok(level, t):
    if abs(t[1]-t[2])<=level*t[3]:
        return True
    return False

We use this function to create a predicate passed through to `itertools.filterfalse` which is then used to obtain an iterator on the anomalies.

In [32]:
from itertools import filterfalse
pred = lambda t: is_ok(5, t)
anomalies = filterfalse(pred, data_with_stats)

We materialize the anomalies...

In [33]:
list(anomalies)#materialize

[(3468, 999999985.2059408, 999999999.9970062, 2.938478635704255),
 (7003, 999999984.4080132, 1000000000.0042406, 2.903339878118392),
 (9461, 1000000016.7163684, 1000000000.0004239, 2.897045435294657)]

## To think of, but not hand in

What kinds of anomalies will this algorithm pick up? What kinds would a shorter "window" of anomaly detection, like 100 points around the time in question pick? How might you create an algorithm which does window based averaging? (hint: the window size is small compared to the time series size). 

Finally think a bit of how you might implement all of this in a production environment..remember that data streaming in might get backed up when you handle an anomaly.

(Some inspiration might accrue if you look at the docs for `collections.deque`).

Answer:  
The algorithm picks up the generated random number that are 5 standard deviation away from the online mean. To implement a shorter window, for example 100, one can record the average and standard deviation of the online mean and standard deviation before and compare to the current online mean and standard deviation to calculate for that only for this time window.